<a href="https://colab.research.google.com/github/JiaxinWang123/SapFlower/blob/main/Sapflow_2020_Pontotoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#### Training BiLSTM model
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import csv
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Define the file path for saving the CSV
csv_file_path = "/content/drive/MyDrive/Sapflow_epoch50/best_rmse_per_clone.csv"

# Define the Bi-LSTM model
class BiLSTM(nn.Module):
    def __init__(self, input_size=8, hidden_layer_size=80, output_size=1):  # Change input size to 8
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, bidirectional=True)  # Bi-directional LSTM

        self.linear = nn.Linear(hidden_layer_size * 2, output_size)  # Double the hidden size for bidirectional LSTM

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))

        return predictions

# Load the data
data_origin = pd.read_csv("/content/Pontotoc_2020_Sapflow_Cleaned_day153_280_removed_outliners.csv")

# Drop NaN values and select relevant columns
selected_columns = ['Date', 'VPD', 'PAR_Den_Avg']  # Remove the clone column from selected columns
subset_data = data_origin[selected_columns].copy()  # Make a copy of the DataFrame

# Define a dictionary to store the best RMSE for each clone
best_rmse_dict = {}

# Iterate over all clones from column index 5 to the end of the dataset
for clone_index in range(5, len(data_origin.columns)):
    clone = data_origin.columns[clone_index]  # Get the clone name from column index
    clone_data = data_origin[['Date', 'VPD', 'PAR_Den_Avg', clone]].copy()  # Make a copy of the DataFrame

    # Drop NaN values for the current clone
    clone_data.dropna(inplace=True)
    clone_data = clone_data[clone_data[clone].values > 0.005]

    # Rename the columns for clarity
    clone_data.columns = ['Date', 'VPD', 'PAR_Den_Avg', clone]

    # Prepare data for training
    data = clone_data
    data['Date'] = pd.to_datetime(data['Date'])

    # Extract hour from the date
    data['Hour'] = data['Date'].dt.hour
    data['Date'] = data['Date'].dt.date

    # Convert date to ordinal
    data['Date'] = data['Date'].apply(lambda x: x.toordinal())

    # Convert hour to one-hot encoding
    data = pd.get_dummies(data, columns=['Hour'], prefix='Hour')

    # Normalize the other columns
    scaler = StandardScaler()
    data['Date'] = scaler.fit_transform(data['Date'].values.reshape(-1,1))
    data['PAR_Den_Avg'] = scaler.fit_transform(data['PAR_Den_Avg'].values.reshape(-1,1))
    data['VPD'] = scaler.fit_transform(data['VPD'].values.reshape(-1,1))

    # Define the input and target variables
    X = data[['Date', 'VPD', 'PAR_Den_Avg'] + [col for col in data.columns if col.startswith('Hour_')]].values
    y = data[clone].values

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    # Convert the data into PyTorch tensors and move them to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X_train_tensor = torch.tensor(X_train).float().to(device)
    y_train_tensor = torch.tensor(y_train).float().to(device)

    X_test_tensor = torch.tensor(X_test).float().to(device)
    y_test_tensor = torch.tensor(y_test).float().to(device)

    # Define the model, loss function, and optimizer and move the model to GPU
    model = BiLSTM(input_size=X.shape[1], hidden_layer_size=80, output_size=1).to(device)
    loss_function = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model multiple times and select the best fit model based on RMSE
    best_rmse = float('inf')
    best_model = None

    num_trainings = 5  # Number of times to train the model
    for _ in range(num_trainings):

        # Train the model
        epochs = 50
        for i in range(epochs):
            optimizer.zero_grad()
            model.hidden = (torch.zeros(2, 1, model.hidden_layer_size).to(device),  # Bidirectional LSTM has 2 layers
                            torch.zeros(2, 1, model.hidden_layer_size).to(device))

            y_pred = model(X_train_tensor)

            loss = loss_function(y_pred, y_train_tensor.unsqueeze(1))
            loss.backward()
            optimizer.step()

            if i % 5 == 1:
                print(f'epoch: {i:3} loss: {loss.item():10.8f}')

        # Evaluate the model
        with torch.no_grad():
            preds = model(X_test_tensor)
            loss = loss_function(preds, y_test_tensor.unsqueeze(1))
            print(f'Test loss: {loss.item()}')

        # Calculate RMSE
        rmse = mean_squared_error(y_test, preds.cpu().numpy(), squared=False)
        print(f'RMSE: {rmse}')

        # Check if current model is the best fit
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model

    # Store the best RMSE for this clone in the dictionary
    best_rmse_dict[clone] = best_rmse

    # Use the best model for predictions
    with torch.no_grad():
        preds = best_model(X_test_tensor)
        loss = loss_function(preds, y_test_tensor.unsqueeze(1))
        print(f'Best model test loss: {loss.item()}')

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(preds.cpu().numpy(), label='Predicted')
    plt.plot(y_test, label='True')
    plt.legend()
    plt.show()

    # Calculate RMSE using the best model
    rmse = mean_squared_error(y_test, preds.cpu().numpy(), squared=False)
    print(f'Best model RMSE: {rmse}')

    # Save the best model state
    torch.save(best_model.state_dict(), f'/content/drive/MyDrive/Sapflow_epoch50/model_{clone}_model_state_dict.pt')

    # Write clone and best RMSE to the CSV file
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Clone', 'Best_RMSE'])  # Write header
        for clone, rmse in best_rmse_dict.items():
            writer.writerow([clone, rmse])
